In [97]:

from time import sleep
from math import ceil
import os
from random import randint
from requests import HTTPError
import pandas as pd
from queries import get_coordinates, get_property_list, get_property_details

In [98]:


# Load the CSV file into a DataFrame
# Load the CSV file into a DataFrame
path = pd.read_csv('neighborhoods.csv')

# Set the index using all columns except 'zips'
neighborhoods_df = path.set_index(['neighborhood', 'neighborhood_ascii', 'lat', 'lng', 'city_name', 'city_id', 'state_name', 'state_id', 'source', 'timezone', 'county_fips', 'county_name', 'id'])

# Split the 'zips' column into separate rows
neighborhoods_df = neighborhoods_df['zips'].str.split(' ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='zip_code')


neighborhoods_df

,neighborhood,lat,lng,city_name,city_id,state_name,state_id,source,timezone,county_fips,county_name,id,level_12,zip_code
0,Atlanta University Center,33.74947,-84.41125,Atlanta,1840013660,Georgia,GA,polygon,America/New_York,13121,Fulton,184001366011100,0,30314
1,Hunter Hills,33.75872,-84.43123,Atlanta,1840013660,Georgia,GA,polygon,America/New_York,13121,Fulton,184001366011101,0,30314
2,Bankhead,33.76901,-84.42452,Atlanta,1840013660,Georgia,GA,polygon,America/New_York,13121,Fulton,184001366011102,0,30318
3,Bankhead,33.76901,-84.42452,Atlanta,1840013660,Georgia,GA,polygon,America/New_York,13121,Fulton,184001366011102,1,30314
4,English Avenue,33.77070,-84.41099,Atlanta,1840013660,Georgia,GA,polygon,America/New_York,13121,Fulton,184001366011103,0,30318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,Buckhead,33.83955,-84.37937,Atlanta,1840013660,Georgia,GA,point,America/New_York,13121,Fulton,184001366014125,0,30305
5456,Blair Village,33.65761,-84.37020,Atlanta,1840013660,Georgia,GA,point,America/New_York,13121,Fulton,184001366014126,0,30354
5457,West Tampa,27.95335,-82.48926,Tampa,1840015982,Florida,FL,point,America/New_York,12057,Hillsborough,184001598214105,0,33607
5458,Terra Cotta,38.95372,-76.99831,Washington,1840006060,District of Columbia,DC,point,America/New_York,11001,District of Columbia,184000606014149,0,20011


In [99]:
selected_neighbor_df = neighborhoods_df[['neighborhood', 'zip_code']]
selected_neighbor_df

,neighborhood,zip_code
0,Atlanta University Center,30314
1,Hunter Hills,30314
2,Bankhead,30318
3,Bankhead,30314
4,English Avenue,30318
...,...,...
5455,Buckhead,30305
5456,Blair Village,30354
5457,West Tampa,33607
5458,Terra Cotta,20011


In [100]:
selected_neighbor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5460 entries, 0 to 5459
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   neighborhood  5460 non-null   object
 1   zip_code      5460 non-null   object
dtypes: object(2)
memory usage: 85.4+ KB


In [16]:
# # Load the CSV file into a DataFrame
# rank_path = pd.read_csv('city_rank.csv')

# # Inspect the columns
# print(rank_path.columns)

# # If 'zips' exists under a different name, update it here
# # Assuming 'zip_codes' is the correct column name
# if 'zip_codes' in rank_path.columns:
#     # Set the index using all columns except the correct zips column
#     city_rank_df = rank_path.set_index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
#                                         'county_name', 'lat', 'lng', 'population', 'density', 'source',
#                                         'military', 'incorporated', 'timezone', 'ranking', 'zip_codes', 'id'])

#     # Split the 'zip_codes' column into separate rows
#     city_rank_df = city_rank_df['zip_codes'].str.split(' ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='ZIP Code')

#     print(city_rank_df)
# else:
#     print("Column 'zips' or equivalent not found in the dataset.")

In [101]:
# Load the CSV file into a DataFrame
rank_path = pd.read_csv('city_rank.csv')

# Set the index using all columns except 'zips'
city_rank_df = rank_path.set_index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'id'])

# Split the 'zips' column into separate rows
city_rank_df = city_rank_df['zips'].str.split(' ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='zip_code')

# Display the resulting DataFrame
city_rank_df

,city,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,id,level_15,zip_code
0,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,1840034016,0,11229
1,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,1840034016,1,11228
2,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,1840034016,2,11226
3,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,1840034016,3,11225
4,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,1840034016,4,11224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48184,Kohatk,AZ,Arizona,4021,Pinal,32.5781,-112.0032,0,77.3,shape,False,False,America/Phoenix,3,1840022983,0,85634
48185,Ironville,PA,Pennsylvania,42013,Blair,40.6586,-78.2155,0,59.2,shape,False,False,America/New_York,3,1840152922,0,16686
48186,Newkirk,NM,New Mexico,35019,Guadalupe,35.0635,-104.2715,0,0.0,shape,False,False,America/Denver,3,1840024978,0,88417
48187,Falcon Village,TX,Texas,48427,Starr,26.5652,-99.1341,0,38.9,shape,False,False,America/Chicago,3,1840018314,0,78545


In [102]:
selected_rank_df = city_rank_df[['ranking', 'zip_code']]
selected_rank_df

,ranking,zip_code
0,1,11229
1,1,11228
2,1,11226
3,1,11225
4,1,11224
...,...,...
48184,3,85634
48185,3,16686
48186,3,88417
48187,3,78545


In [103]:
selected_rank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48189 entries, 0 to 48188
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ranking   48189 non-null  int64 
 1   zip_code  48189 non-null  object
dtypes: int64(1), object(1)
memory usage: 753.1+ KB


In [104]:
realtor_path = pd.read_csv('realtor_data.csv')
realtor_path

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2226377,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354.0,3600.0,2022-03-25
2226378,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354.0,1616.0,2022-03-25
2226379,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354.0,3200.0,2022-03-24
2226380,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354.0,933.0,2022-03-24


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        float64
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(8), object(4)
memory usage: 203.8+ MB


In [120]:
realtor_path['zip_code'] = realtor_path['zip_code'].astype(str).str.zfill(5)


In [106]:
realtor_path['zip_code'] = realtor_path['zip_code'].str.replace('.0', '', regex=False)


In [121]:
realtor_path

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,00601,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,00601,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,00795,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,00731,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,00680,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2226377,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354,3600.0,2022-03-25
2226378,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354,1616.0,2022-03-25
2226379,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354,3200.0,2022-03-24
2226380,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354,933.0,2022-03-24


In [122]:
realtor_path.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        object 
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(7), object(5)
memory usage: 203.8+ MB


In [124]:

# Now merge the DataFrames
merged_df_1 = pd.merge(selected_neighbor_df, realtor_path, on='zip_code')

# Display the merged DataFrame
merged_df_1

,neighborhood,zip_code,brokered_by,status,price,bed,bath,acre_lot,street,city,state,house_size,prev_sold_date
0,Atlanta University Center,30314,21417.0,for_sale,549900.0,4.0,4.0,0.10,1729649.0,Atlanta,Georgia,1870.0,2020-12-30
1,Atlanta University Center,30314,6916.0,for_sale,125000.0,NaN,NaN,0.08,1423689.0,Atlanta,Georgia,NaN,2020-07-02
2,Atlanta University Center,30314,22611.0,for_sale,180000.0,2.0,1.0,0.15,348737.0,Atlanta,Georgia,1158.0,NaN
3,Atlanta University Center,30314,78186.0,for_sale,69900.0,NaN,NaN,0.07,1743414.0,Atlanta,Georgia,NaN,NaN
4,Atlanta University Center,30314,78186.0,for_sale,69900.0,NaN,NaN,0.07,1743413.0,Atlanta,Georgia,NaN,2022-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174329,Southwest Waterfront,20024,16829.0,sold,1249000.0,4.0,4.0,0.03,1592848.0,Washington,District of Columbia,2440.0,2021-12-08
1174330,Southwest Waterfront,20024,16829.0,sold,454900.0,1.0,1.0,NaN,1569139.0,Washington,District of Columbia,1000.0,2021-12-10
1174331,Southwest Waterfront,20024,22611.0,sold,499000.0,2.0,2.0,NaN,244513.0,Washington,District of Columbia,1194.0,2021-12-27
1174332,Southwest Waterfront,20024,59064.0,sold,1190000.0,4.0,4.0,0.03,1455794.0,Washington,District of Columbia,2440.0,2021-12-27


In [125]:
merged_df = pd.merge(selected_rank_df, merged_df_1, on='zip_code')
merged_df 

,ranking,zip_code,neighborhood,brokered_by,status,price,bed,bath,acre_lot,street,city,state,house_size,prev_sold_date
0,1,11229,Gerritsen Beach,23592.0,for_sale,999000.0,2.0,1.0,NaN,1450948.0,New York City,New York,840.0,NaN
1,1,11229,Gerritsen Beach,84536.0,for_sale,659000.0,2.0,2.0,0.04,1808463.0,Brooklyn,New York,980.0,2014-10-01
2,1,11229,Gerritsen Beach,81944.0,for_sale,938000.0,3.0,3.0,0.05,384285.0,Brooklyn,New York,2352.0,2021-07-07
3,1,11229,Gerritsen Beach,81286.0,for_sale,748000.0,3.0,1.0,0.04,628929.0,Brooklyn,New York,1224.0,2011-01-24
4,1,11229,Gerritsen Beach,76342.0,for_sale,538000.0,3.0,1.0,0.04,1053053.0,Brooklyn,New York,1000.0,1996-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638125,3,85339,Laveen,53339.0,sold,440000.0,3.0,2.0,0.18,1421021.0,Laveen,Arizona,1996.0,2022-01-19
1638126,3,85339,Laveen,107243.0,sold,339000.0,3.0,2.0,0.14,1302003.0,Laveen,Arizona,1552.0,2022-01-19
1638127,3,85339,Laveen,66216.0,sold,439900.0,5.0,3.0,0.15,1360935.0,Laveen,Arizona,2462.0,2022-01-06
1638128,3,85339,Laveen,30687.0,sold,395000.0,3.0,2.0,0.13,1476747.0,Laveen,Arizona,1864.0,2022-01-06


In [126]:
merged_df.isnull().sum()

ranking                0
zip_code               0
neighborhood           0
brokered_by         1438
status                 0
price                111
bed               120206
bath              110460
acre_lot          453791
street             12563
city                  35
state                  0
house_size        226029
prev_sold_date    361273
dtype: int64

In [127]:
merged_df['prev_sold_date'].fillna(value='other', inplace=True)

In [128]:
merged_df.isnull().sum()

ranking                0
zip_code               0
neighborhood           0
brokered_by         1438
status                 0
price                111
bed               120206
bath              110460
acre_lot          453791
street             12563
city                  35
state                  0
house_size        226029
prev_sold_date         0
dtype: int64

In [129]:
cleaned_df = merged_df.dropna()
cleaned_df

,ranking,zip_code,neighborhood,brokered_by,status,price,bed,bath,acre_lot,street,city,state,house_size,prev_sold_date
1,1,11229,Gerritsen Beach,84536.0,for_sale,659000.0,2.0,2.0,0.04,1808463.0,Brooklyn,New York,980.0,2014-10-01
2,1,11229,Gerritsen Beach,81944.0,for_sale,938000.0,3.0,3.0,0.05,384285.0,Brooklyn,New York,2352.0,2021-07-07
3,1,11229,Gerritsen Beach,81286.0,for_sale,748000.0,3.0,1.0,0.04,628929.0,Brooklyn,New York,1224.0,2011-01-24
4,1,11229,Gerritsen Beach,76342.0,for_sale,538000.0,3.0,1.0,0.04,1053053.0,Brooklyn,New York,1000.0,1996-12-16
9,1,11229,Gerritsen Beach,34944.0,for_sale,678000.0,3.0,2.0,0.04,1567742.0,Brooklyn,New York,1216.0,2011-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638124,3,85339,Laveen,84534.0,sold,458000.0,4.0,3.0,0.23,1675725.0,Laveen,Arizona,2056.0,2022-01-19
1638125,3,85339,Laveen,53339.0,sold,440000.0,3.0,2.0,0.18,1421021.0,Laveen,Arizona,1996.0,2022-01-19
1638126,3,85339,Laveen,107243.0,sold,339000.0,3.0,2.0,0.14,1302003.0,Laveen,Arizona,1552.0,2022-01-19
1638127,3,85339,Laveen,66216.0,sold,439900.0,5.0,3.0,0.15,1360935.0,Laveen,Arizona,2462.0,2022-01-06


In [130]:
cleaned_df.describe()

,ranking,brokered_by,price,bed,bath,acre_lot,street,house_size
count,1.012403e+06,1.012403e+06,1.012403e+06,1.012403e+06,1.012403e+06,1.012403e+06,1.012403e+06,1.012403e+06
mean,1.433617e+00,5.333029e+04,7.337206e+05,3.376095e+00,2.575950e+00,2.384008e+01,9.928176e+05,2.005633e+03
std,7.651784e-01,3.093934e+04,1.527879e+06,1.624061e+00,1.563390e+00,5.334405e+02,5.215439e+05,4.043389e+03
min,1.000000e+00,2.000000e+00,9.500000e+02,1.000000e+00,1.000000e+00,0.000000e+00,5.681000e+03,1.000000e+02
25%,1.000000e+00,2.418600e+04,2.800000e+05,3.000000e+00,2.000000e+00,6.000000e-02,5.428870e+05,1.238000e+03
50%,1.000000e+00,5.292800e+04,4.690000e+05,3.000000e+00,2.000000e+00,1.300000e-01,1.046816e+06,1.670000e+03
75%,2.000000e+00,7.922100e+04,7.820000e+05,4.000000e+00,3.000000e+00,2.000000e-01,1.426027e+06,2.346000e+03
max,4.000000e+00,1.101400e+05,5.150000e+08,1.080000e+02,1.750000e+02,1.000000e+05,1.999512e+06,1.560780e+06


In [131]:
cleaned_df.isnull().sum()

ranking           0
zip_code          0
neighborhood      0
brokered_by       0
status            0
price             0
bed               0
bath              0
acre_lot          0
street            0
city              0
state             0
house_size        0
prev_sold_date    0
dtype: int64

In [132]:
cleaned_df.duplicated().sum()


90844

In [133]:
duplicates = cleaned_df.duplicated()

# Display only the rows that are duplicates
duplicate_rows = cleaned_df[duplicates]

duplicate_rows

,ranking,zip_code,neighborhood,brokered_by,status,price,bed,bath,acre_lot,street,city,state,house_size,prev_sold_date
1615,1,11229,Gerritsen Beach,84536.0,for_sale,659000.0,2.0,2.0,0.04,1808463.0,Brooklyn,New York,980.0,2014-10-01
1616,1,11229,Gerritsen Beach,81944.0,for_sale,938000.0,3.0,3.0,0.05,384285.0,Brooklyn,New York,2352.0,2021-07-07
1617,1,11229,Gerritsen Beach,81286.0,for_sale,748000.0,3.0,1.0,0.04,628929.0,Brooklyn,New York,1224.0,2011-01-24
1618,1,11229,Gerritsen Beach,76342.0,for_sale,538000.0,3.0,1.0,0.04,1053053.0,Brooklyn,New York,1000.0,1996-12-16
1623,1,11229,Gerritsen Beach,34944.0,for_sale,678000.0,3.0,2.0,0.04,1567742.0,Brooklyn,New York,1216.0,2011-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638124,3,85339,Laveen,84534.0,sold,458000.0,4.0,3.0,0.23,1675725.0,Laveen,Arizona,2056.0,2022-01-19
1638125,3,85339,Laveen,53339.0,sold,440000.0,3.0,2.0,0.18,1421021.0,Laveen,Arizona,1996.0,2022-01-19
1638126,3,85339,Laveen,107243.0,sold,339000.0,3.0,2.0,0.14,1302003.0,Laveen,Arizona,1552.0,2022-01-19
1638127,3,85339,Laveen,66216.0,sold,439900.0,5.0,3.0,0.15,1360935.0,Laveen,Arizona,2462.0,2022-01-06


In [134]:
# Drop duplicate rows
cleaned_df_no_duplicates = cleaned_df.drop_duplicates()

# Display the DataFrame without duplicates
cleaned_df_no_duplicates

,ranking,zip_code,neighborhood,brokered_by,status,price,bed,bath,acre_lot,street,city,state,house_size,prev_sold_date
1,1,11229,Gerritsen Beach,84536.0,for_sale,659000.0,2.0,2.0,0.04,1808463.0,Brooklyn,New York,980.0,2014-10-01
2,1,11229,Gerritsen Beach,81944.0,for_sale,938000.0,3.0,3.0,0.05,384285.0,Brooklyn,New York,2352.0,2021-07-07
3,1,11229,Gerritsen Beach,81286.0,for_sale,748000.0,3.0,1.0,0.04,628929.0,Brooklyn,New York,1224.0,2011-01-24
4,1,11229,Gerritsen Beach,76342.0,for_sale,538000.0,3.0,1.0,0.04,1053053.0,Brooklyn,New York,1000.0,1996-12-16
9,1,11229,Gerritsen Beach,34944.0,for_sale,678000.0,3.0,2.0,0.04,1567742.0,Brooklyn,New York,1216.0,2011-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637722,4,85339,Laveen,84534.0,sold,458000.0,4.0,3.0,0.23,1675725.0,Laveen,Arizona,2056.0,2022-01-19
1637723,4,85339,Laveen,53339.0,sold,440000.0,3.0,2.0,0.18,1421021.0,Laveen,Arizona,1996.0,2022-01-19
1637724,4,85339,Laveen,107243.0,sold,339000.0,3.0,2.0,0.14,1302003.0,Laveen,Arizona,1552.0,2022-01-19
1637725,4,85339,Laveen,66216.0,sold,439900.0,5.0,3.0,0.15,1360935.0,Laveen,Arizona,2462.0,2022-01-06


In [135]:
cleaned_df_no_duplicates = cleaned_df_no_duplicates[['state', 'city', 'street', 'neighborhood', 'bed', 'bath', 'acre_lot', 'house_size', 'price', 'ranking',	'zip_code', 'brokered_by', 'status', 'prev_sold_date']]
cleaned_df_no_duplicates.head(20)

,state,city,street,neighborhood,bed,bath,acre_lot,house_size,price,ranking,zip_code,brokered_by,status,prev_sold_date
1,New York,Brooklyn,1808463.0,Gerritsen Beach,2.0,2.0,0.04,980.0,659000.0,1,11229,84536.0,for_sale,2014-10-01
2,New York,Brooklyn,384285.0,Gerritsen Beach,3.0,3.0,0.05,2352.0,938000.0,1,11229,81944.0,for_sale,2021-07-07
3,New York,Brooklyn,628929.0,Gerritsen Beach,3.0,1.0,0.04,1224.0,748000.0,1,11229,81286.0,for_sale,2011-01-24
4,New York,Brooklyn,1053053.0,Gerritsen Beach,3.0,1.0,0.04,1000.0,538000.0,1,11229,76342.0,for_sale,1996-12-16
9,New York,Brooklyn,1567742.0,Gerritsen Beach,3.0,2.0,0.04,1216.0,678000.0,1,11229,34944.0,for_sale,2011-07-26
10,New York,Brooklyn,513483.0,Gerritsen Beach,4.0,4.0,0.07,2400.0,2488000.0,1,11229,81944.0,for_sale,2007-10-25
15,New York,Brooklyn,116782.0,Gerritsen Beach,3.0,2.0,0.08,1047.0,699999.0,1,11229,81286.0,for_sale,2016-09-09
16,New York,Brooklyn,726042.0,Gerritsen Beach,3.0,2.0,0.05,1092.0,699999.0,1,11229,81286.0,for_sale,1997-04-04
19,New York,Brooklyn,1309064.0,Gerritsen Beach,2.0,1.0,0.04,656.0,535000.0,1,11229,48297.0,for_sale,other
20,New York,Brooklyn,919169.0,Gerritsen Beach,4.0,2.0,0.04,1360.0,689000.0,1,11229,96696.0,for_sale,2001-03-23
